In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
from ipywidgets import *
from IPython.display import HTML
import time
import ffmpeg 
from matplotlib.animation import FuncAnimation
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d
import uproot
mpl.rcParams['animation.embed_limit'] = 2**128
%matplotlib notebook

In [2]:
# Open data file
fIn = uproot.open('S2-tuple.root')

In [3]:
# Select trees of interest
clusterTree = fIn['DefaultReconstruction/clusters']
muonTree = fIn['DefaultReconstruction/muons']
runinfoTree = fIn['DefaultReconstruction/runInfos']

# Select leaves of interest
clusterTreeEntries=clusterTree.arrays(['start', 'rejected', 'MuonTag', 'NSTag', 'ESTag', 'MuonTreePos', 'chanID',
                                       'chanActive', 'cubes', 'wfTreePos', 'chanAmplitude', 'onionLayer'])
muonTreeEntries = muonTree.arrays(['htime','startCube','endCube','type','stopped', 'mx', 'cx', 'my', 'cy'])
runTreeEntries = runinfoTree.arrays(['runNumber', 'runDuration', 'MuonsRate'])

In [4]:
#Make sure these are floating point values:
scale_x = 5.0
scale_y = 1.6
scale_z = 1.6

#Axes are scaled down to fit in scene
max_scale=max(scale_x, scale_y, scale_z)

scale_x=scale_x/max_scale
scale_y=scale_y/max_scale
scale_z=scale_z/max_scale

#Create scaling matrix
scale = np.array([[scale_x,0,0,0],[0,scale_y,0,0], [0,0,scale_z,0], [0,0,0,0.6]])

def get_proj_scale(self):
    """
    Create the projection matrix from the current viewing position.
    elev stores the elevation angle in the z plane
    azim stores the azimuth angle in the x,y plane
    dist is the distance of the eye viewing point from the object point.
    """
    relev, razim = np.pi * self.elev/180, np.pi * self.azim/180

    xmin, xmax = self.get_xlim3d()
    ymin, ymax = self.get_ylim3d()
    zmin, zmax = self.get_zlim3d()

    # transform to uniform world coordinates 0-1.0,0-1.0,0-1.0
    worldM = proj3d.world_transformation(xmin, xmax, ymin, ymax, zmin, zmax)

    # look into the middle of the new coordinates
    R = np.array([1, 0.4, -0.3])

    xp = R[0] + np.cos(razim) * np.cos(relev) * self.dist
    yp = R[1] + np.sin(razim) * np.cos(relev) * self.dist
    zp = R[2] + np.sin(relev) * self.dist
    E = np.array((xp, yp, zp))

    self.eye = E
    self.vvec = R - E
    self.vvec = self.vvec / proj3d.mod(self.vvec)

    if abs(relev) > np.pi/2:
        # upside down
        V = np.array((0, 0, -1))
    else:
        V = np.array((0, 0, 1))
    zfront, zback = -self.dist, self.dist

    viewM = proj3d.view_transformation(E, R, V)
    perspM = proj3d.persp_transformation(zfront, zback)
    M0 = np.dot(viewM, worldM)
    M = np.dot(perspM, M0)

    return np.dot(M, scale);

Axes3D.get_proj=get_proj_scale

In [5]:
measureTime = 1.8

timer = 0  
for iEntry in range(len(clusterTreeEntries['start'])):
    if clusterTreeEntries['start'][iEntry] * 25e-9 < measureTime:
        timer += 1

print('for a measure time of {:.2f} seconds of detector time, a minimum of {:.0f} frames are required'.format(measureTime, timer))

for a measure time of 1.80 seconds of detector time, a minimum of 977 frames are required


In [6]:
def track(iEntry):
    # prepare some coordinates
    matrix = [[ [0 for col in range(16)] for col in range(16)] for row in range(50)]

    for i in range(len(clusterTreeEntries['cubes'][iEntry])):
        matrix[int(clusterTreeEntries['cubes'][iEntry][i][2])][int(clusterTreeEntries['cubes'][iEntry][i][0])][int(clusterTreeEntries['cubes'][iEntry][i][1])] = 1
    
    return matrix

In [7]:
%%capture
x_min_anim, x_max_anim = 0, 50
y_min_anim, y_max_anim = 0, 16
z_min_anim, z_max_anim = 0, 16

events_fig = plt.figure(figsize=(6, 7))
gs = gridspec.GridSpec(nrows=1, ncols=1)
ax_an_event = events_fig.add_subplot(gs[0, 0],projection='3d',xlabel="Z (Plane)",ylabel="X (Cube)",
                                     zlabel="Y (Cube)", xlim=(x_min_anim, x_max_anim),
                                     ylim=(y_min_anim, y_max_anim), zlim=(z_min_anim, z_max_anim))
ax_an_event.view_init(30, -130) 

In [8]:
def animate_events(frame, entries2, time, nframes, filled, perc):    
    events_fig.suptitle(r'Visualisation of {:.2f} seconds detector time'.format(time))    
    
    if not filled:
        #we start with a clear plot
        ax_an_event.clear()
    
    if entries2[frame] != 0:
        
        #calculate track
        matrix = track(entries2[frame])
        
        #draw event
        if clusterTreeEntries['MuonTag'][entries2[frame]]:
            ax_an_event.voxels(np.array(matrix), facecolors='#0000FF75', edgecolor='#08298A75')
        elif clusterTreeEntries['ESTag'][entries2[frame]]:
            ax_an_event.voxels(np.array(matrix), facecolors='#FF000075', edgecolor='#DF010175')
        else:
            ax_an_event.voxels(np.array(matrix), facecolors='#80FF0075', edgecolor='#29870875')
            
    else:
        matrix = [[ [0 for col in range(16)] for col in range(16)] for row in range(50)]
        #the colors are of no importance here and are chosen arbitrarily
        ax_an_event.voxels(np.array(matrix), facecolors='blue', edgecolor='darkblue') 
    
    # show calculation progress
    load.value += 1/nframes*100
    perc += 1/nframes*100
    print('{:.2f}%'.format(load.value), end="\r")
    frame+=1
    
    return [events_fig]

In [9]:
def anim(a):
    load.value = 0  # loading bar value
    perc = 0  # percent done
    frame = 0 #which frame are we on
    
    #we start with a clear plot
    ax_an_event.clear()
    
    #create a data list
    time_axis = np.linspace(0, time_anim.value, frames_anim.value)
    entries = []
    entries2 = np.zeros(frames_anim.value, dtype=int)
    for iEntry in range(len(clusterTreeEntries['start'])):
        if clusterTreeEntries['start'][iEntry] * 25e-9 < time_anim.value:
            entries.append(iEntry)

    for iEntry in entries:
        for i in range(len(time_axis) - 1):
            if (time_axis[i] <= clusterTreeEntries['start'][iEntry] * 25e-9) and (clusterTreeEntries['start'][iEntry] * 25e-9 <= time_axis[i+1]):
                entries2[i] = iEntry
  
    anim = FuncAnimation(events_fig, animate_events, frames=frames_anim.value, 
                         fargs=(entries2, time_anim.value, frames_anim.value, filled.value, perc), blit=True, interval=50)
    
    #should the animation be displayed or saved
    if disp.value:
        display(HTML(anim.to_jshtml()))
    else:
        anim.save('{}.mp4'.format(save.value))

In [10]:
# interactive settings for animation
style = {'description_width': 'initial'}
time_anim = FloatSlider(description="time", min=0, max=5, step=0.01, value=0.20)
frames_anim = IntSlider(description="frames", min=500, max=3000, value=1000)

param = VBox([time_anim, frames_anim])

disp = Dropdown(options=[("Display", True), ("Save", False)], Value=True, description='Output')
filled = Dropdown(options=[("Without filled", False), ("With filled", True)], Value=False, description='Filled')
save = Text(description='Save name', value='detectorAnimation')
output = VBox([filled, disp, save])

start_anim = Button(description='Create animation')
start_anim.on_click(anim)

load = FloatProgress(min=0, max=100, description='Loading:', style=style)

children = [param, output]
tab = Tab()
tab.children = children
tab.set_title(0, 'Parameters')
tab.set_title(1, 'Output')

ui = VBox([tab, start_anim, load])

In [11]:
display(ui)

A Jupyter Widget